In [ ]:
import os
import glob
import warnings
import h5py as h5
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from copy import deepcopy

from gensit.experiments import *
from gensit.physics_models import *
from gensit.config import Config
from gensit.inputs import Inputs
from gensit.outputs import Outputs
from gensit.utils.misc_utils import *
from gensit.utils.math_utils import *
from gensit.utils.probability_utils import *

In [ ]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

In [ ]:
# Specify experiment id
config_id = "generic/synthetic_data_simulation.toml"
config_dir = f'../data/inputs/configs/{config_id}'
relative_config_dir = os.path.relpath(config_dir,os.getcwd())

In [ ]:
# Create new logging object
logger = setup_logger(
    __name__,
    console_level = 'INFO',
    file_level = 'EMPTY'
)

# Read config
config = Config(
    path = relative_config_dir,
    settings = None,
    console_level = 'INFO',
    logger = logger
)

# Update root
config.path_sets_root()

In [ ]:
# Prepare inputs
inputs = Inputs(
    config = config,
    synthetic_data = True,
    logger = logger
)

# Pass inputs to device
inputs.pass_to_device() 

intensity_model = instantiate_intensity_model(
    config = config,
    **vars(inputs.data),
    logger = logger
)
# Get and remove config
config = pop_variable(intensity_model,'config',config)

# Build the physics model
logger.hilight("Initializing the physics model ...")
physics_model = instantiate_physics_model(
    config = config,
    trial = None,
    intensity_model = intensity_model,
    true_parameters = inputs.true_parameters,
    logger = logger
)

In [ ]:
time_series = []
seeds = [1234, 5678, 91011]
for i in range(3):
    theta_sample = {"alpha":torch.tensor(1.2),"beta":torch.tensor(0.7)}
    init_destination_attraction = torch.full((inputs.data.dims['destination'],1), 1./inputs.data.dims['destination'])
    n_runs = 10
    with tqdm(total=n_runs) as pbar:    
        # Solve SDE
        destination_attraction_samples = physics_model.run(
            init_destination_attraction = init_destination_attraction,
            free_parameters = theta_sample,
            n_iterations = n_runs,
            dt = config['harris_wilson_model']['dt'],
            requires_grad = True,
            generate_time_series = True,
            seed = seeds[i],
            semaphore = None,
            samples = None,
            pbar = pbar
        )
        time_series.append(destination_attraction_samples)

In [ ]:
# Create a plot for each dimension D
plt.figure(figsize=(10, 6))
time_series_index = 1
J = np.shape(time_series[time_series_index])[1]
for d in range(inputs.data.dims['destination']):
    plt.plot(time_series[time_series_index][:, d], label=f'Destination {d+1}')

# Labeling the plot
plt.xlabel('Time Steps (T)')
plt.ylabel('Value')
plt.title('Time Series across Dimensions')
plt.legend(loc='best')
plt.show()

In [ ]:
np.save('../../times_series.npy',np.array([ts.numpy() for ts in time_series]))